In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9977
2,Huelva,Confirmados PDIA 14 días,1454
3,Huelva,Tasa PDIA 14 días,"284,68329472944316"
4,Huelva,Confirmados PDIA 7 días,568
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,122
630,Municipio de Huelva sin especificar,Curados,84


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9977.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2798.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 887 personas en los últimos 7 días 

Un positivo PCR cada 335 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9977.0,568.0,1454.0,510743.0,111.210531,284.683295,120.0
Huelva-Costa,5520.0,365.0,960.0,288115.0,126.685525,333.200285,85.0
Huelva (capital),2798.0,162.0,429.0,143663.0,112.763899,298.615510,36.0
Condado-Campiña,3083.0,170.0,403.0,155057.0,109.637101,259.904422,32.0
Lepe,881.0,55.0,179.0,27431.0,200.503080,652.546389,19.0
Isla Cristina,333.0,55.0,93.0,21264.0,258.653123,437.358916,13.0
Moguer,325.0,38.0,90.0,22088.0,172.039116,407.461065,10.0
Gibraleón,347.0,12.0,17.0,12607.0,95.185215,134.845721,7.0
Punta Umbría,224.0,12.0,31.0,15242.0,78.729825,203.385382,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,82.0,16.0,32.0,3371.0,474.636606,949.273213,1.0
Lepe,881.0,55.0,179.0,27431.0,200.503080,652.546389,19.0
Palma del Condado (La),452.0,27.0,63.0,10761.0,250.906050,585.447449,5.0
Niebla,112.0,4.0,24.0,4117.0,97.158125,582.948749,0.0
Villanueva de los Castillejos,58.0,4.0,14.0,2820.0,141.843972,496.453901,2.0
San Bartolomé de la Torre,41.0,14.0,18.0,3736.0,374.732334,481.798715,1.0
Villalba del Alcor,98.0,6.0,16.0,3338.0,179.748352,479.328939,0.0
Isla Cristina,333.0,55.0,93.0,21264.0,258.653123,437.358916,13.0
Palos de la Frontera,206.0,19.0,49.0,11289.0,168.305430,434.050846,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Nerva,86.0,1.0,10.0,5235.0,19.102197,191.021968,0.0,0.100000
Minas de Riotinto,64.0,1.0,6.0,3848.0,25.987526,155.925156,0.0,0.166667
Niebla,112.0,4.0,24.0,4117.0,97.158125,582.948749,0.0,0.166667
Aroche,51.0,2.0,11.0,3073.0,65.082981,357.956394,0.0,0.181818
Cerro de Andévalo (El),36.0,1.0,5.0,2364.0,42.301184,211.505922,0.0,0.200000
Calañas,44.0,1.0,5.0,2791.0,35.829452,179.147259,0.0,0.200000
Bollullos Par del Condado,231.0,3.0,13.0,14272.0,21.020179,91.087444,0.0,0.230769
Bonares,42.0,2.0,8.0,6058.0,33.014196,132.056784,1.0,0.250000
Villanueva de los Castillejos,58.0,4.0,14.0,2820.0,141.843972,496.453901,2.0,0.285714
